## Read in Data

In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames, CategoricalArrays, Statistics

In [2]:
data = CSV.read("team_data_with_averages.csv", DataFrame);

In [3]:
Random.seed!(8008)
rows = round(Int, .7 * nrow(data))
sample = randperm(nrow(data))[1:rows];
train = data[sample,:];
test = data[Not(sample),:];

In [4]:
data

Row,Tm,year,W,WIN.,avg_G_C,avg_GS_C,avg_MP_C,avg_FG_C,avg_FGA_C,avg_FG_Percentage_C,avg_3P_C,avg_3PA_C,avg_3P_Percentage_C,avg_2P_C,avg_2PA_C,avg_2P_Percentage_C,avg_eFG_Percentage_C,avg_FT_C,avg_FTA_C,avg_FT_Percentage_C,avg_ORB_C,avg_DRB_C,avg_TRB_C,avg_AST_C,avg_STL_C,avg_BLK_C,avg_TOV_C,avg_PF_C,avg_PTS_C,avg_G_PG,avg_GS_PG,avg_MP_PG,avg_FG_PG,avg_FGA_PG,avg_FG_Percentage_PG,avg_3P_PG,avg_3PA_PG,avg_3P_Percentage_PG,avg_2P_PG,avg_2PA_PG,avg_2P_Percentage_PG,avg_eFG_Percentage_PG,avg_FT_PG,avg_FTA_PG,avg_FT_Percentage_PG,avg_ORB_PG,avg_DRB_PG,avg_TRB_PG,avg_AST_PG,avg_STL_PG,avg_BLK_PG,avg_TOV_PG,avg_PF_PG,avg_PTS_PG,avg_G_PF,avg_GS_PF,avg_MP_PF,avg_FG_PF,avg_FGA_PF,avg_FG_Percentage_PF,avg_3P_PF,avg_3PA_PF,avg_3P_Percentage_PF,avg_2P_PF,avg_2PA_PF,avg_2P_Percentage_PF,avg_eFG_Percentage_PF,avg_FT_PF,avg_FTA_PF,avg_FT_Percentage_PF,avg_ORB_PF,avg_DRB_PF,avg_TRB_PF,avg_AST_PF,avg_STL_PF,avg_BLK_PF,avg_TOV_PF,avg_PF_PF,avg_PTS_PF,avg_G_SF,avg_GS_SF,avg_MP_SF,avg_FG_SF,avg_FGA_SF,avg_FG_Percentage_SF,avg_3P_SF,avg_3PA_SF,avg_3P_Percentage_SF,avg_2P_SF,avg_2PA_SF,avg_2P_Percentage_SF,avg_eFG_Percentage_SF,avg_FT_SF,avg_FTA_SF,avg_FT_Percentage_SF,avg_ORB_SF,avg_DRB_SF,avg_TRB_SF,avg_AST_SF,avg_STL_SF,⋯
,String3,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,MIL,2020,56,0.767,60.5,32.0,20.5,3.15,7.0,0.4645,1.0,3.15,0.32,2.15,3.85,0.5725,0.5315,1.0,1.35,0.6725,0.8,2.7,3.45,1.15,0.45,1.55,0.95,1.85,8.25,38.0,18.6667,19.0667,3.3,7.16667,0.425,1.0,2.73333,0.342667,2.3,4.4,0.481333,0.496333,1.46667,1.9,0.714667,0.7,2.26667,2.96667,3.36667,0.666667,0.166667,1.46667,1.4,9.1,48.0,21.6667,18.6333,4.86667,9.5,0.471667,0.933333,2.93333,0.310667,3.96667,6.56667,0.578333,0.533333,2.53333,3.86667,0.709,1.23333,5.76667,6.93333,2.43333,0.5,0.5,1.6,1.8,13.2,42.0,13.75,16.7,3.2,6.875,0.45575,1.225,3.2,0.28725,1.975,3.65,0.5265,0.5375,1.2,1.475,0.7425,0.55,2.675,3.275,1.7,0.525,⋯
2,TOR,2020,53,0.736,30.0,20.6667,19.2667,3.2,6.8,0.379,0.9,2.36667,0.266667,2.23333,4.36667,0.417333,0.433,1.0,1.3,0.823333,1.13333,4.33333,5.46667,1.63333,0.533333,0.566667,1.16667,2.36667,8.2,34.6667,33.3333,25.0667,4.23333,10.0667,0.475333,1.96667,5.26667,0.414,2.26667,4.76667,0.534,0.574333,2.76667,3.23333,0.901333,0.333333,2.66667,2.96667,4.93333,1.06667,0.333333,1.76667,2.06667,13.2,42.6667,17.6667,17.8333,3.76667,8.4,0.391667,0.933333,2.83333,0.265333,2.83333,5.53333,0.468,0.439667,2.0,2.46667,0.809667,1.0,3.4,4.33333,1.46667,0.533333,0.666667,1.1,1.73333,10.3667,39.2,16.0,17.18,1.94,4.28,0.4172,0.46,1.44,0.2678,1.46,2.82,0.4892,0.481,0.7,1.02,0.6222,0.84,2.06,2.92,1.22,0.74,⋯
3,LAL,2020,52,0.732,61.5,31.5,18.0,3.0,4.35,0.686,0.0,0.1,0.55,2.95,4.3,0.6885,0.691,1.1,2.05,0.5735,2.15,4.45,6.6,0.65,0.45,1.35,1.0,2.8,7.15,51.0,16.0,21.0,4.15,8.875,0.4445,1.125,3.125,0.35275,3.025,5.725,0.49775,0.51075,1.45,2.025,0.6965,0.475,3.0,3.475,4.625,0.825,0.2,1.85,1.275,10.75,37.3333,20.6667,20.2,4.63333,9.66667,0.4715,0.833333,2.63333,0.316,3.83333,7.0,0.5385,0.5175,2.93333,3.56667,0.7915,1.06667,3.66667,4.73333,1.56667,0.633333,0.933333,1.36667,1.5,13.0667,40.0,1.0,7.8,0.5,1.3,0.4,0.4,0.9,0.471,0.1,0.4,0.25,0.56,0.1,0.1,1.0,0.1,1.0,1.1,0.5,0.3,⋯
4,BOS,2020,48,0.667,31.8,12.8,12.94,2.26,3.8,0.6086,0.08,0.32,0.321333,2.14,3.48,0.6236,0.6188,0.74,1.

In [5]:
C_indices = 5:29
PG_indices = 30:54
PF_indices = 55:79
SF_indices = 80:104
SG_indices = 105:129;

In [6]:
Xtrain = train[:,5:129]
XtrainC = train[:,C_indices]
XtrainPG = train[:,PG_indices]
XtrainPF = train[:,PF_indices]
XtrainSF = train[:,SF_indices]
XtrainSG = train[:,SG_indices]
ytrain = train[:,4]

165-element Vector{Float64}:
 0.89
 0.232
 0.712
 0.463
 0.451
 0.611
 0.354
 0.256
 0.5
 0.61
 0.598
 0.695
 0.606
 ⋮
 0.297
 0.333
 0.524
 0.573
 0.756
 0.476
 0.515
 0.585
 0.354
 0.72
 0.549
 0.427

In [7]:
Xtest = test[:,5:129]
XtestC = test[:,C_indices]
XtestPG = test[:,PG_indices]
XtestPF = test[:,PF_indices]
XtestSF = test[:,SF_indices]
XtestSG = test[:,SG_indices]
ytest = test[:,4];

In [8]:
function pCor(threshold, X)
    n,p = size(X)
    temp = cor(X)
    corPairs = Vector{Tuple{Int, Int}}()
    for i in 1:p
        for j in i+1:p
            if abs(temp[i,j]) > threshold
                push!(corPairs, (i,j))
            end
        end
    end
    return corPairs
end

function newData(X)
    n,p = size(X)
    newX = X[:,1]
    for i in 1:p
        if i != 1
            newX = hcat(newX, X[:,i]) # add not transformed variable first
        end
        newX = hcat(newX, X[:,i].^2) # add squared
        newX = hcat(newX, broadcast(abs, X[:,i]).^.5) # add square root
        newX = hcat(newX, broadcast(log, broadcast(abs, X[:,i]) .+ 1)) # add log + 1
    end
    return newX
end

function calculate_predictions(X, coefficients)
    return Matrix(X) * coefficients
end

function calculate_mse(y_true, y_pred)
    squared_error = (y_true .- y_pred) .^ 2
    return mean(squared_error)
end

function calculate_mae(y_true, y_pred)
    absolute_error = abs.(y_true .- y_pred)
    return mean(absolute_error)
end

function calculate_osr2(y_true, y_pred)
    ssr = sum((y_true - y_pred) .^ 2)
    tss = sum((y_true .- mean(y_true)) .^ 2)
    return 1 - ssr / tss
end

calculate_osr2 (generic function with 1 method)

## Holistic Regression

In [75]:
function holistic(y, X, lambda, threshold, k)
    # Setting initial model
    model = JuMP.Model(Gurobi.Optimizer)
    n,p = size(X)
    # create the variables
    @variable(model, betas[1:p])
    @variable(model, a[1:p] >= 0)
    @variable(model, z[1:p], Bin)
    @objective(model, Min, sum((y[i] - sum(X[i,:].*betas))^2 for i=1:n) + lambda * sum(a[i] for i=1:p))
    # for abs value for l1 norm
    @constraint(model, [i=1:p], betas[i] <= a[i])
    @constraint(model, [i=1:p], betas[i] >= -a[i])
    # for sparsity
    @constraint(model, [i=1:p], betas[i] <= 100*z[i])
    @constraint(model, [i=1:p], betas[i] >= -100*z[i])
    @constraint(model, sum(z[i] for i in 1:p) <= k)
    # for group sparsity
    for i in 0:24
        @constraint(model, sum(z[j] for j in (4*i+1):(4*(i+1))) <= 1)
    end
    # for pairwise colinearity
    pairs = pCor(threshold, X)
    for pair in pairs
        @constraint(model, z[pair[1]] + z[pair[2]] <= 1)
    end
        
    optimize!(model)

    return value.(betas)
end

holistic (generic function with 1 method)

In [76]:
newXtrain = newData(Xtrain)
newXtrainC = newData(XtrainC)
newXtestC = newData(XtestC);
newXtrainPG = newData(XtrainPG)
newXtestPG = newData(XtestPG);
newXtrainPF = newData(XtrainPF)
newXtestPF = newData(XtestPF);
newXtrainSG = newData(XtrainSG)
newXtestSG = newData(XtestSG);
newXtrainSF = newData(XtrainSF)
newXtestSF = newData(XtestSF);
newXtest = newData(Xtest)

71×500 Matrix{Float64}:
 61.5     3782.25   7.84219  4.13517  31.5     …   43.2306  2.56418  2.02485
 55.3333  3061.78   7.43864  4.03129  39.6667      64.6416  2.83549  2.20166
 20.0      400.0    4.47214  3.04452   4.0        117.361   3.2914   2.47092
 47.5     2256.25   6.89202  3.88156  40.5         52.7076  2.69444  2.11142
 74.0     5476.0    8.60233  4.31749  51.0         44.5556  2.5836   2.03797
 42.6     1814.76   6.52687  3.77506  16.8     …   85.2544  3.03864  2.32565
 79.0     6241.0    8.88819  4.38203  52.5        132.826   3.39485  2.52773
 59.0     3481.0    7.68115  4.09434  27.3333     111.654   3.25064  2.44813
 73.5     5402.25   8.57321  4.3108   41.0         24.4036  2.22261  1.78171
 61.5     3782.25   7.84219  4.13517  41.0        166.41    3.59166  2.63189
 76.5     5852.25   8.74643  4.35028  39.5     …   19.1406  2.09165  1.68176
 56.5     3192.25   7.51665  4.05178  13.0         92.16    3.09839  2.36085
 57.5     3306.25   7.58288  4.06903  40.0        11

In [77]:
C_coeff = holistic(ytrain, newXtrainC, 1, .5, 10); # implement the function with lambda = 1, p = 0.5, and k = 8

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8650U CPU @ 1.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2487 rows, 300 columns and 5122 nonzeros
Model fingerprint: 0x7948f585
Model has 5050 quadratic objective terms
Variable types: 200 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 5e+05]
  QObjective range [5e+00, 3e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 46.9057890
Presolve removed 2044 rows and 0 columns
Presolve time: 0.03s
Presolved: 443 rows, 300 columns, 2212 nonzeros
Presolved model has 5050 quadratic obje

In [78]:
# Calculate predictions
pred = calculate_predictions(newXtestC, C_coeff)
# Calculate metrics
mse = calculate_mse(ytest, pred)
mae = calculate_mae(ytest, pred)
osr2 = calculate_osr2(ytest, pred)
# Print results
println("OSR2: ", osr2)
println("MAE: ", mae)
println("MSE: ", mse)

OSR2: 0.12382498019838217
MAE: 0.10688283407110818
MSE: 0.01769288983001332


The first model is pretty bad. Let's see if we normalizing the data helps.

In [98]:
lambda_values = [0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.8,0.9]
k_values = [10,11,12,13,14,15,16,17,18,19,20]
rho_values = [0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.8,0.9]
# Best hyperparameters and corresponding MAE
best_hyperparameters = (lambda=0, k=0, rho=0)
best_mae = Inf
# Grid search
for lambda in lambda_values
    for k in k_values
        for rho in rho_values
            println("Running iteration with λ=$lambda, k=$k, ρ=$rho")
            betas = holistic(ytrain, newXtrainC, lambda, rho, k);
            y_pred = newXtestC * betas  # Make predictions on validation set
            error = calculate_mae(ytest, y_pred)  # Calculate MAE
            if error < best_mae
                best_mae = error
                best_hyperparameters = (lambda=lambda, k=k, rho=rho)
            end
        end
    end
end

Running iteration with λ=0.2, k=10, ρ=0.2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8650U CPU @ 1.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3539 rows, 300 columns and 7226 nonzeros
Model fingerprint: 0x356b8429
Model has 5050 quadratic objective terms
Variable types: 200 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 5e+05]
  QObjective range [5e+00, 3e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 46.9057890
Presolve removed 3101 rows and 0 columns
Presolve time: 0.02s
Presolved: 438 rows, 300 columns, 1813 nonzer

Excessive output truncated after 526649 bytes.

In [99]:
println("Best hyperparameters: λ=$(best_hyperparameters.lambda), k=$(best_hyperparameters.k), ρ=$(best_hyperparameters.rho) with MAE=$best_mae")

Best hyperparameters: λ=0.35, k=11, ρ=0.9 with MAE=0.09752441475844194


In [100]:
C_coeff = holistic(ytrain, newXtrainC, .35, .9, 11); # implement the function with lambda = 1, p = 0.5, and k = 8

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8650U CPU @ 1.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 738 rows, 300 columns and 1624 nonzeros
Model fingerprint: 0x5c7b1978
Model has 5050 quadratic objective terms
Variable types: 200 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e-01, 5e+05]
  QObjective range [5e+00, 3e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 46.9057890
Presolve removed 294 rows and 0 columns
Presolve time: 0.00s
Presolved: 444 rows, 300 columns, 1141 nonzeros
Presolved model has 5050 quadratic object

In [101]:
# Calculate predictions
pred = calculate_predictions(newXtestC, C_coeff)
# Calculate metrics
mse = calculate_mse(ytest, pred)
mae = calculate_mae(ytest, pred)
osr2 = calculate_osr2(ytest, pred)
# Print results
println("OSR2: ", osr2)
println("MAE: ", mae)
println("MSE: ", mse)

OSR2: 0.2569157931356
MAE: 0.09752441475844194
MSE: 0.015005343349610043


## XGBoost

In [9]:
temp = IAI.XGBoostRegressor(random_seed = 1234, max_categoric_levels_before_warning = 100)
grid_xgb = IAI.GridSearch(temp, 
                          num_estimators = [50, 100, 150], 
                          max_depth = 7:10, 
                          learning_rate = [0.01, 0.05, 0.1], 
                          min_child_weight = [1, 3, 5], 
                          gamma = [0.1, 0.2], 
                          subsample = [0.8, 0.9, 1.0], 
                          colsample_bytree = [0.8, 0.9, 1.0], 
                          criterion = :mse)
IAI.fit!(grid_xgb, XtrainC, Array(ytrain), validation_criterion = :mse);

In [10]:
XGBmodel = IAI.get_learner(grid_xgb)

Fitted XGBoostRegressor

In [11]:
ypred = IAI.predict(XGBmodel, XtestC);

In [12]:
# Calculate metrics
mse = calculate_mse(ytest, ypred)
mae = calculate_mae(ytest, ypred)
osr2 = calculate_osr2(ytest, ypred)
# Print results
println("OSR2: ", osr2)
println("MAE: ", mae)
println("MSE: ", mse)

OSR2: -0.2157562678252254
MAE: 0.12793114295140118
MSE: 0.024550165458552094


## Random Forest

In [88]:
temp = IAI.RandomForestRegressor(random_seed = 1234, max_categoric_levels_before_warning = 100)
grid_rf = IAI.GridSearch(temp, num_trees = 100, minbucket = 1:10, criterion = :mse)
IAI.fit!(grid_rf, XtrainC, Array(ytrain), validation_criterion = :mse);

In [89]:
RFmodel = IAI.get_learner(grid_rf)

Fitted RandomForestRegressor

In [92]:
ypred = IAI.predict(RFmodel, XtestC);

In [93]:
# Calculate metrics
mse = calculate_mse(ytest, ypred)
mae = calculate_mae(ytest, ypred)
osr2 = calculate_osr2(ytest, ypred)
# Print results
println("OSR2: ", osr2)
println("MAE: ", mae)
println("MSE: ", mse)

OSR2: 0.001075587534341338
MAE: 0.116462570666208
MSE: 0.02017160861567061


## Normalizing the Data

In [19]:
function detect_outliers_iqr(data)
    q1 = quantile(data, 0.25)
    q3 = quantile(data, 0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return findall((data .< lower_bound) .| (data .> upper_bound))
end

# Apply to a column
outliers_list = []
for i in 5:29
    outliers = detect_outliers_iqr(data[:, i])
    push!(outliers_list, outliers)
end
outliers_list

Let's normalize using Z-score since we have a fair amount of outliers.

In [22]:
function z_score_normalize(column)
    mean_val = mean(column)
    std_dev = std(column)
    return (column .- mean_val) ./ std_dev
end

# Normalize columns except the target
for col in 5:129
    data[:, col] = z_score_normalize(data[!, col])
end

In [23]:
data

Row,Tm,year,W,WIN.,avg_G_C,avg_GS_C,avg_MP_C,avg_FG_C,avg_FGA_C,avg_FG_Percentage_C,avg_3P_C,avg_3PA_C,avg_3P_Percentage_C,avg_2P_C,avg_2PA_C,avg_2P_Percentage_C,avg_eFG_Percentage_C,avg_FT_C,avg_FTA_C,avg_FT_Percentage_C,avg_ORB_C,avg_DRB_C,avg_TRB_C,avg_AST_C,avg_STL_C,avg_BLK_C,avg_TOV_C,avg_PF_C,avg_PTS_C,avg_G_PG,avg_GS_PG,avg_MP_PG,avg_FG_PG,avg_FGA_PG,avg_FG_Percentage_PG,avg_3P_PG,avg_3PA_PG,avg_3P_Percentage_PG,avg_2P_PG,avg_2PA_PG,avg_2P_Percentage_PG,avg_eFG_Percentage_PG,avg_FT_PG,avg_FTA_PG,avg_FT_Percentage_PG,avg_ORB_PG,avg_DRB_PG,avg_TRB_PG,avg_AST_PG,avg_STL_PG,avg_BLK_PG,avg_TOV_PG,avg_PF_PG,avg_PTS_PG,avg_G_PF,avg_GS_PF,avg_MP_PF,avg_FG_PF,avg_FGA_PF,avg_FG_Percentage_PF,avg_3P_PF,avg_3PA_PF,avg_3P_Percentage_PF,avg_2P_PF,avg_2PA_PF,avg_2P_Percentage_PF,avg_eFG_Percentage_PF,avg_FT_PF,avg_FTA_PF,avg_FT_Percentage_PF,avg_ORB_PF,avg_DRB_PF,avg_TRB_PF,avg_AST_PF,avg_STL_PF,avg_BLK_PF,avg_TOV_PF,avg_PF_PF,avg_PTS_PF,avg_G_SF,avg_GS_SF,avg_MP_SF,avg_FG_SF,avg_FGA_SF,avg_FG_Percentage_SF,avg_3P_SF,avg_3PA_SF,avg_3P_Percentage_SF,avg_2P_SF,avg_2PA_SF,avg_2P_Percentage_SF,avg_eFG_Percentage_SF,avg_FT_SF,avg_FTA_SF,avg_FT_Percentage_SF,avg_ORB_SF,avg_DRB_SF,avg_TRB_SF,avg_AST_SF,avg_STL_SF,⋯
,String3,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,MIL,2020,56,0.767,0.568752,0.233748,0.229019,-0.054139,0.347217,-0.738289,2.77111,3.28869,0.668562,-0.787888,-0.870844,0.669984,0.129556,-0.599223,-0.66058,0.0402355,-1.55808,-0.890055,-1.19082,-0.0494337,-0.32024,2.0224,-0.428343,-0.6373,0.0597801,-1.14704,-0.778769,-0.772857,-0.292595,-0.47459,0.128718,-0.00460904,-0.0705352,0.17257,-0.363856,-0.596841,0.435051,0.381708,-0.34173,-0.289143,-1.01192,1.10513,0.151512,0.399467,-0.527316,-0.726648,-0.182894,-0.448082,-0.723846,-0.279587,-0.437598,-0.354799,-0.404312,0.993793,0.774208,0.137804,0.859729,1.09723,0.457129,0.761095,0.314022,1.35236,0.590222,1.13618,1.68783,-0.0723021,-0.223023,1.67916,1.16605,1.73598,-0.318511,-0.138298,1.2921,-0.230288,1.11597,-0.685252,-0.94429,-0.834556,0.0861133,-0.0856771,0.631401,0.680496,0.492123,-0.537203,-0.175471,-0.386368,0.795406,0.765871,-0.208916,-0.30924,-0.0251708,-0.465543,-0.0307929,-0.106305,0.22964,-0.678209,⋯
2,TOR,2020,53,0.736,-1.73883,-0.651549,-0.0373343,-0.0116158,0.259227,-2.0776,2.43597,2.30889,0.378459,-0.713989,-0.626115,-1.64103,-1.38465,-0.599223,-0.700955,1.70836,-1.01557,0.292499,-0.11696,0.647281,0.0683688,-0.904429,0.0725933,0.559652,0.0437117,-1.37323,0.173357,0.404676,0.368002,0.472068,1.15804,1.80828,1.84893,1.26484,-0.392915,-0.439919,1.35408,1.85846,1.01599,0.906018,1.66374,-0.669968,0.708651,0.399467,0.587425,0.554714,1.34716,0.0588024,0.861451,0.743534,-0.816117,-0.624278,-0.559521,0.207194,0.381761,-1.32323,0.859729,1.01524,0.0231036,-0.070319,-0.0760396,-0.654617,-1.15151,0.527692,0.37049,1.13636,-0.631644,-0.145519,-0.359407,0.26528,-0.155367,0.429719,0.0423467,-0.36539,0.35605,-0.858796,-0.795356,-0.74906,-0.941402,-1.11071,-0.0621433,-1.11073,-1.09377,-0.81689,-0.663948,-0.792474,0.241157,-0.0888138,-0.823699,-0.778787,-1.30445,0.487403,-0.697684,-0.427219,-0.411011,0.0410406,⋯
3,LAL,2020,52,0.732,0.64441,0.19469,-0.310886,-0.181709,-0.818656,2.7314,-0.580324,-0.526276,1.91963,-0.0784637,-0.657693,2.39766,2.5815,-0.470806

In [24]:
train = data[sample,:];
test = data[Not(sample),:];
XtrainC = train[:,C_indices]
XtrainPG = train[:,PG_indices]
XtrainPF = train[:,PF_indices]
XtrainSF = train[:,SF_indices]
XtrainSG = train[:,SG_indices]
ytrain = train[:,4]
XtestC = test[:,C_indices]
XtestPG = test[:,PG_indices]
XtestPF = test[:,PF_indices]
XtestSF = test[:,SF_indices]
XtestSG = test[:,SG_indices]
ytest = test[:,4];
newXtrainC = newData(XtrainC)
newXtestC = newData(XtestC);
newXtrainPG = newData(XtrainPG)
newXtestPG = newData(XtestPG);
newXtrainPF = newData(XtrainPF)
newXtestPF = newData(XtestPF);
newXtrainSG = newData(XtrainSG)
newXtestSG = newData(XtestSG);
newXtrainSF = newData(XtrainSF)
newXtestSF = newData(XtestSF);

In [26]:
lambda_values = [0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.8,0.9]
k_values = [10,11,12,13,14,15,16,17,18,19,20]
rho_values = [0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.8,0.9]
# Best hyperparameters and corresponding MAE
best_hyperparameters = (lambda=0, k=0, rho=0)
best_mae = Inf
# Grid search
for lambda in lambda_values
    for k in k_values
        for rho in rho_values
            println("Running iteration with λ=$lambda, k=$k, ρ=$rho")
            betas = holistic(ytrain, newXtrainC, lambda, rho, k);
            y_pred = newXtestC * betas  # Make predictions on validation set
            error = calculate_mae(ytest, y_pred)  # Calculate MAE
            if error < best_mae
                best_mae = error
                best_hyperparameters = (lambda=lambda, k=k, rho=rho)
            end
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8650U CPU @ 1.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1226 rows, 300 columns and 2600 nonzeros
Model fingerprint: 0x51300a51
Model has 5050 quadratic objective terms
Variable types: 200 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 2e+02]
  QObjective range [5e-02, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 44.8445480
Presolve removed 750 rows and 0 columns
Presolve time: 0.01s
Presolved: 476 rows, 300 columns, 1585 nonzeros
Presolved model has 5050 quadratic objective terms
Variable types: 200 continuous, 100 integer (100 binary)

Root relaxation: objective 3.719108e+00

In [60]:
size(newXtrain)

(165, 500)

(165, 100)